In [1]:
from utils.PinConfig import PinConfig
import casadi
import pinocchio as pin
import numpy as np

config_file = 'config/SimpleConfig.yaml'
model, data, cmodel, cdata = PinConfig(config_file)
print(model)

q_waypoints = np.array([[0, np.pi/4, 0, 0, 0, 0], 
                      [-np.pi/6, np.pi/6, np.pi/6, -np.pi/6, -np.pi/3, -np.pi/4],
                      [-np.pi/4, np.pi/4, np.pi/3, 0, -np.pi/6, -np.pi/2]])

q0 = q_waypoints[0]
print(f"Robot Initial Configuration:{q0}")

pin.forwardKinematics(model, data, q0)
pin.updateFramePlacements(model, data)
end_effector_frame_id = model.getFrameId('tool0')
print("End-Effector Position:", data.oMf[end_effector_frame_id].translation)

pin.forwardKinematics(model, data, q_waypoints[1])
pin.updateFramePlacements(model, data)
end_effector_frame_id = model.getFrameId('tool0')
print("End-Effector Position:", data.oMf[end_effector_frame_id].translation)

Nb joints = 7 (nq=6,nv=6)
  Joint 0 universe: parent=0
  Joint 1 shoulder_pan_joint: parent=0
  Joint 2 shoulder_lift_joint: parent=1
  Joint 3 elbow_joint: parent=2
  Joint 4 wrist_1_joint: parent=3
  Joint 5 wrist_2_joint: parent=4
  Joint 6 wrist_3_joint: parent=5

Robot Initial Configuration:[0.         0.78539816 0.         0.         0.         0.        ]
End-Effector Position: [ 0.50734912  0.2329     -0.48584621]
End-Effector Position: [ 0.47226412 -0.06123615 -0.43286983]


### Collision Sphere Generation

In [2]:
from utils.ObstacleSpheres import *

config_file = 'config/SimpleConfig.yaml'
obs = ObsConfig(config_file)

obs.spheres_from_robot(q0 = q_waypoints[0].astype(float).tolist())  # Get initial robot obstacle spheres
obs.spheres_from_env()


cq = casadi.SX.sym('q', model.nq)
get_obstacle_distances = casadi.Function(
    'get_obstacle_distances', 
    [cq], 
    [casadi.vertcat(*obstacle_distances(obs.env_sphere_list, obs.robot_sphere_list, cmodel, cdata, cq))]
    )


print(get_obstacle_distances(q_waypoints[0]))

[0.386406, 0.398906, 0.0263761, 0.21097, 0.397441, 0.411705, 0.255841, 0.320138, 0.120666, 0.245574, 0.0485242, 0.216655, 0.174939, 0.274533, 0.353169, 0.38866, 0.416639, 0.42415, 0.000958329, 0.193827, -0.0353814, 0.313021, -0.0346953, 0.200853, -0.0501069, 0.222211, -0.0493436, 0.266332, -0.0168213, 0.195295, -0.0527015, 0.244365, -0.030209, 0.320729, -0.0444081, 0.209383, -0.0136932, 0.334915, -0.022534, 0.326075, -0.00317207, 0.364977, -0.0133521, 0.327561, -0.0268813, 0.251936, -0.0221185, 0.256699, -0.0224938, 0.280648, -0.0317169, 0.245969, 0.0011788, 0.278772]


In [3]:
ngc = model.nq   # Number of joints
ndof = model.nv   # Number of DOF
nx = 2*ngc    # Number of states
cx = casadi.SX.sym("x", 2*ngc, 1)
ctau = casadi.SX.sym("tau", ndof, 1)
cacc = cpin.aba(cmodel, cdata, cx[:ngc], cx[ngc:], ctau) # Forward dynamics

cq = casadi.SX.sym("x", model.nq, 1)


# Joint limits
lowerVelocityLimit = np.pi/2 * np.array([-1, -1, -1, -1, -1, -1])  # rad/s
upperVelocityLimit = np.pi/2 * np.array([1, 1, 1, 1, 1, 1])
lowerTorqueLimit = 100 * np.array([-1, -1, -1, -1, -1, -1]) 
upperTorqueLimit = 100 * np.array([1, 1, 1, 1, 1, 1])
print(model.upperPositionLimit)
print(model.lowerPositionLimit)


q0 = q_waypoints[0]
qT = q_waypoints[-1]
tau0 = pin.rnea(model, data, q0, np.zeros(ndof), np.zeros(ndof))
tauT = pin.rnea(model, data, qT, np.zeros(ndof), np.zeros(ndof))
print(tau0)
print(tauT)

Feq = casadi.Function(
    "Dyn",
    [cx, ctau],
    [casadi.vertcat(cx[ngc:], cacc)],
)

[6.28318531 6.28318531 3.14159265 6.28318531 6.28318531 6.28318531]
[-6.28318531 -6.28318531 -3.14159265 -6.28318531 -6.28318531 -6.28318531]
[  0.         -36.08572402  -9.26563407   0.97299901  -0.09997149
   0.        ]
[ 8.88178420e-16 -2.17616448e+01  5.05844512e+00  1.31084531e+00
 -1.18267539e-01  0.00000000e+00]


In [4]:
from utils.Lagrange import *
from utils.LGL_collocation import LGL_collocation
from utils.LocalChart import *

Ntp = 10
Np = q_waypoints.shape[0]-1  # Number of phase (dersired waypoints: N+1)
T = Np*Ntp  # Number of total collocation points / optimization variables

LGL_Pts, LGL_Wts = Normalize_LGL_Pts_Wts(Ntp)
DL,_ = differentiation_matrix(Ntp, LGL_Pts)

LGL = LGL_collocation(Ntp, ngc, ndof)
StatesDiff, StateDiff, LocalsDiff, LocalDiff= LGL.getDiffFuncs()

### Initial Guesses

In [5]:
t_initial = 1
ini_collo_pts = t_initial/2 * (LGL_Pts+1)
ini_q = np.full((Ntp, ngc), 0.0)
ini_x = [None] * T
ini_u = [None] * T

for k in range(Np):
    for j in range(Ntp):
        ini_q[k] = q_waypoints[k] + ini_collo_pts[j]/t_initial*(q_waypoints[k+1]-q_waypoints[k])

    ini_dq = np.array([np.squeeze(StateDiff(casadi.vertcat(*ini_q), t_initial, DL[i,:])) for i in range(Ntp)])
    ini_ddq = np.array([np.squeeze(StateDiff(casadi.vertcat(*ini_dq), t_initial, DL[i,:])) for i in range(Ntp)])
    ini_tau = np.array([pin.rnea(model, data, ini_q[i], ini_dq[i], ini_ddq[i]) for i in range(Ntp)])  # M@ddq + C@dq + G

    for j in range(Ntp):
        t = k*Ntp + j
        ini_x[t] =  np.concatenate([ini_q[j],ini_dq[j]])
        ini_u[t] = np.array(ini_tau[j])

### Nonlinear Programming Problem (NLP)

In [6]:
opti = casadi.Opti()
# Optimization variables
var_xs = [opti.variable(2*ngc) for _ in range(T)]
var_us = [opti.variable(ndof) for _ in range(T)]
var_deltaT = [opti.variable(1) for _ in range(Np)]

# Constraints
for i in range(Np):
    opti.subject_to(var_deltaT[i] >= 0.0)

    for tp in range(Ntp):
        t = i*Ntp + tp
        # dynamics
        opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[tp,:]) 
                        == Feq(var_xs[t], var_us[t]))
        
        # joint constraints
        opti.subject_to(var_xs[t][:ngc] <= model.upperPositionLimit)
        opti.subject_to(var_xs[t][:ngc] >= model.lowerPositionLimit)
        # joint velocity constraints
        opti.subject_to(var_xs[t][ngc:] <= upperVelocityLimit)
        opti.subject_to(var_xs[t][ngc:] >= lowerVelocityLimit)
        # torque constraints
        opti.subject_to(var_us[t] <= upperTorqueLimit)
        opti.subject_to(var_us[t] >= lowerTorqueLimit)

        # collision avoidance
        opti.subject_to(get_obstacle_distances(var_xs[t][:ngc]) >= 0.005)

# waypoints constraints (N-1 middle waypoints)
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp][:ngc] == q_waypoints[i])

# continuity
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp-1] == var_xs[i*Ntp])
    opti.subject_to(var_us[i*Ntp-1] == var_us[i*Ntp])

# continuity of derivatives
for i in range(Np-1):
    opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:]) 
                    == StatesDiff(casadi.vertcat(*var_xs[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i+1], DL[0,:]))
    
    opti.subject_to(LocalDiff(casadi.vertcat(*var_us[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:])
                   == LocalDiff(casadi.vertcat(*var_us[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i], DL[0,:]))


# boundary conditions
opti.subject_to(var_xs[0][:ngc] == q0)
opti.subject_to(var_xs[-1][:ngc] == qT)
opti.subject_to(var_xs[0][ngc:] == 0)
opti.subject_to(var_xs[-1][ngc:] == 0)
#opti.subject_to(var_us[0] == tau0)
#opti.subject_to(var_us[-1] == tauT)

# Objective
totalcost = 0

R = np.eye(ndof)
for i in range(Np):
    for tp in range(Ntp):
        var_xs_Phase = var_xs[i*Ntp: (i+1)*Ntp]
        dq = [dq_Phase[ngc:] for dq_Phase in var_xs_Phase]
        ddq = StateDiff(casadi.vertcat(*dq), var_deltaT[i], DL[tp,:])
        totalcost += var_deltaT[i]/2 * LGL_Wts[tp] * casadi.mtimes([ddq.T, R, ddq])
        
    totalcost += var_deltaT[i]

opti.minimize(totalcost)
opti.solver("ipopt") 


for t in var_deltaT:
    opti.set_initial(t, t_initial)
for t in range(T):
    opti.set_initial(var_xs[t], ini_x[t])
    opti.set_initial(var_us[t], ini_u[t])

In [7]:
try:
    sol = opti.solve_limited()
    sol_xs = [opti.value(var_x) for var_x in var_xs]
    sol_us = [opti.value(var_u) for var_u in var_us]
    sol_T = [opti.value(var_t) for var_t in var_deltaT]
except:
    print("ERROR in convergence, plotting debug info.")
    sol_xs = [opti.debug.value(var_x) for var_x in var_xs]
    sol_us = [opti.debug.value(var_u) for var_u in var_us]



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.2.

Number of nonzeros in equality constraint Jacobian...:     5256
Number of nonzeros in inequality constraint Jacobian.:     4042
Number of nonzeros in Lagrangian Hessian.............:     5882

Total number of variables............................:      362
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      306
Total number of inequality c